# Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
import edurel.utils.duckdb as ddbu
import edurel.utils.yaml as yu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"


In [3]:
# con.close()

In [4]:
# con = ddbu.con_file(f"{DB_DIR}/adw-oltp/adw-oltp.duckdb")

In [5]:
con = ddbu.con_mem()
SQL_DIR = f"{DB_DIR}/company_en"
ddbu.exe_sql_file(con, f"{SQL_DIR}/schema.sql")
ddbu.exe_sql_file(con, f"{SQL_DIR}/data.sql")

In [6]:
ddbu.tablenames(con)

['EmpProj', 'Employee', 'OrgUnit', 'Project']

In [7]:
ddbu.columns(con, "OrgUnit")

[{'columnname': 'OUID', 'type': 'INTEGER', 'nullable': False},
 {'columnname': 'Head', 'type': 'INTEGER', 'nullable': True},
 {'columnname': 'SuperUnit', 'type': 'INTEGER', 'nullable': True},
 {'columnname': 'Name', 'type': 'VARCHAR', 'nullable': False}]

In [8]:
ddbu.primary_keys(con, "OrgUnit")

['OUID']

In [9]:
additional_fks = {"OrgUnit": ["Employee|Head|EID", "OrgUnit|SuperUnit|OUID"]}

In [10]:
schema = ddbu.schema_yaml(con, additional_fks)
schema

{'tables': [{'tablename': 'EmpProj',
   'columns': [{'columnname': 'EID', 'type': 'INTEGER', 'nullable': False},
    {'columnname': 'PID', 'type': 'INTEGER', 'nullable': False},
    {'columnname': 'NoOfHoursPerWeek', 'type': 'INTEGER', 'nullable': False}],
   'primary_key': ['EID', 'PID'],
   'foreign_keys': [{'fkname': 'fk_EmpProj_Employee_EID_1',
     'sourcecolumns': ['EID'],
     'targettable': 'Employee',
     'targetcolumns': ['EID']},
    {'fkname': 'fk_EmpProj_Project_PID_1',
     'sourcecolumns': ['PID'],
     'targettable': 'Project',
     'targetcolumns': ['PID']}]},
  {'tablename': 'Employee',
   'columns': [{'columnname': 'EID', 'type': 'INTEGER', 'nullable': False},
    {'columnname': 'OUID', 'type': 'INTEGER', 'nullable': False},
    {'columnname': 'LastName', 'type': 'VARCHAR', 'nullable': False},
    {'columnname': 'Hiredate', 'type': 'DATE', 'nullable': False},
    {'columnname': 'Salary', 'type': 'DECIMAL(9,2)', 'nullable': False},
    {'columnname': 'Bonus', 'type':

In [11]:
schema1 = yu.yaml_remove_tags(schema, ["nullable", "fkname"])
schema1

{'tables': [{'tablename': 'EmpProj',
   'columns': [{'columnname': 'EID', 'type': 'INTEGER'},
    {'columnname': 'PID', 'type': 'INTEGER'},
    {'columnname': 'NoOfHoursPerWeek', 'type': 'INTEGER'}],
   'primary_key': ['EID', 'PID'],
   'foreign_keys': [{'sourcecolumns': ['EID'],
     'targettable': 'Employee',
     'targetcolumns': ['EID']},
    {'sourcecolumns': ['PID'],
     'targettable': 'Project',
     'targetcolumns': ['PID']}]},
  {'tablename': 'Employee',
   'columns': [{'columnname': 'EID', 'type': 'INTEGER'},
    {'columnname': 'OUID', 'type': 'INTEGER'},
    {'columnname': 'LastName', 'type': 'VARCHAR'},
    {'columnname': 'Hiredate', 'type': 'DATE'},
    {'columnname': 'Salary', 'type': 'DECIMAL(9,2)'},
    {'columnname': 'Bonus', 'type': 'DECIMAL(9,2)'}],
   'primary_key': ['EID'],
   'foreign_keys': [{'sourcecolumns': ['OUID'],
     'targettable': 'OrgUnit',
     'targetcolumns': ['OUID']}]},
  {'tablename': 'OrgUnit',
   'columns': [{'columnname': 'OUID', 'type': 'INTEG

In [12]:
yu.yaml_print(schema1)

tables:
- tablename: EmpProj
  columns:
  - columnname: EID
    type: INTEGER
  - columnname: PID
    type: INTEGER
  - columnname: NoOfHoursPerWeek
    type: INTEGER
  primary_key:
  - EID
  - PID
  foreign_keys:
  - sourcecolumns:
    - EID
    targettable: Employee
    targetcolumns:
    - EID
  - sourcecolumns:
    - PID
    targettable: Project
    targetcolumns:
    - PID
- tablename: Employee
  columns:
  - columnname: EID
    type: INTEGER
  - columnname: OUID
    type: INTEGER
  - columnname: LastName
    type: VARCHAR
  - columnname: Hiredate
    type: DATE
  - columnname: Salary
    type: DECIMAL(9,2)
  - columnname: Bonus
    type: DECIMAL(9,2)
  primary_key:
  - EID
  foreign_keys:
  - sourcecolumns:
    - OUID
    targettable: OrgUnit
    targetcolumns:
    - OUID
- tablename: OrgUnit
  columns:
  - columnname: OUID
    type: INTEGER
  - columnname: Head
    type: INTEGER
  - columnname: SuperUnit
    type: INTEGER
  - columnname: Name
    type: VARCHAR
  primary_key:
  -

In [13]:
mermaid = yu.yaml_to_mermaid(schema1)
print(mermaid)

erDiagram
    EmpProj {
        INTEGER EID PK
        INTEGER PID PK
        INTEGER NoOfHoursPerWeek
    }
    Employee {
        INTEGER EID PK
        INTEGER OUID
        VARCHAR LastName
        DATE Hiredate
        DECIMAL Salary
        DECIMAL Bonus
    }
    OrgUnit {
        INTEGER OUID PK
        INTEGER Head
        INTEGER SuperUnit
        VARCHAR Name
    }
    Project {
        INTEGER PID PK
        VARCHAR Title
        DECIMAL Budget
    }
    EmpProj }o--|| Employee : "EID -> EID"
    EmpProj }o--|| Project : "PID -> PID"
    Employee }o--|| OrgUnit : "OUID -> OUID"
    OrgUnit }o--|| Employee : "Head -> EID"
    OrgUnit }o--|| OrgUnit : "SuperUnit -> OUID"
